![display relevant image here](path/url/to/image)
- Banner/header image

# Title
- Relevant to Data and Business Context

## Overview
- BLUF (Bottom Line Up Front)
- One paragraph summary of findings and analysis
- Frame your 'story'

## Business Understanding
- Set the stage for analysis
- Why are these findings relevant/important?
- Introduce stakeholders
- Postulate about questions you want to ask/answer

## Data Understanding
- Present the source of data
- Describe the data available
- What is relevant to keep what is not
- Present any data cleaning that needs to happen
- Null values? Type mismatches? Duplicates?

In [9]:
# Imports
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-20.jdk/Contents/Home' 
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col, isnan, when, count, desc,trim, regexp_replace, to_date


findspark.init()
conf = SparkConf().setAppName("Google PlayStore App Analysis").setMaster("local[*]")

sc = SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()
file_path = "google play store apps dataset.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)
df.createOrReplaceTempView("sales_data")


In [10]:
# --- Statistical Analysis ---

# Descriptive statistics for numeric columns
df.describe().show()

# Value distribution for Ratings
spark.sql("""
SELECT Rating, COUNT(*) as num_apps
FROM sales_data
WHERE Rating IS NOT NULL
GROUP BY Rating
ORDER BY Rating
""").show()

# Distribution of app types (Free vs Paid)
spark.sql("""
SELECT Type, COUNT(*) as count
FROM sales_data
GROUP BY Type
""").show()

# Top-rated apps
spark.sql("""
SELECT App, Rating, Reviews
FROM sales_data
WHERE Rating IS NOT NULL
ORDER BY Rating DESC, Reviews DESC
LIMIT 10
""").show()

# Average price of paid apps
spark.sql("""
SELECT AVG(CAST(REPLACE(Price, '$', '') AS DOUBLE)) as avg_price
FROM sales_data
WHERE Type = 'Paid'
""").show()

# App count by category
spark.sql("""
SELECT Category, COUNT(*) as num_apps
FROM sales_data
GROUP BY Category
ORDER BY num_apps DESC
""").show()

# App count by content rating
spark.sql("""
SELECT `Content Rating`, COUNT(*) as count
FROM sales_data
GROUP BY `Content Rating`
ORDER BY count DESC
""").show()


# --- Data Quality Assessment ---

from pyspark.sql.functions import col, isnan, when, count

# Null values per column
df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()

# Check for duplicate records
print("Duplicate rows:", df.count() - df.dropDuplicates().count())

# Show inferred schema (data type consistency)
df.printSchema()

# Count of unique values per column (example: Category, Type, Content Rating)
for column in ["Category", "Type", "Content Rating", "Genres"]:
    spark.sql(f"""
    SELECT `{column}`, COUNT(*) as count
    FROM sales_data
    GROUP BY `{column}`
    ORDER BY count DESC
    """).show()


spark.sql("""
SELECT App, Rating
FROM sales_data
WHERE Rating > 5
""").show()

df.columns

+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|summary|                 App|     Category|     Rating|          Reviews|              Size|          Installs| Type|             Price|Content Rating|Genres|     Last Updated|  Current Ver|       Android Ver|
+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|  count|               10841|        10841|      10841|            10841|             10841|             10841|10841|             10841|         10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|          1.9|        NaN|444225.1924709356|              NULL| 2.866666666666667|  NaN|               0.0|          NULL|  NU

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|  1474|      0|   0|       0|   1|    0|             1|     0|           0|          8|          3|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



Duplicate rows: 483
root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+-------------------+-----+
|           Category|count|
+-------------------+-----+
|             FAMILY| 1972|
|               GAME| 1144|
|              TOOLS|  843|
|            MEDICAL|  463|
|           BUSINESS|  460|
|       PRODUCTIVITY|  424|
|    PERSONALIZATION|  392|
|      COMMUNICATION|  387|
|             SPORTS|  384|
|          LIFESTYLE|  382|
|            FINANCE|  366|
| HEALTH_AND_FITNESS|  340|
|        PHOTOGRAPHY|  335|
|          

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

## Data Preparation



In [3]:

# Data Prep Code Here
df_cleaned = df \
    .withColumn("App", trim(col("App"))) \
    .withColumn("Category", trim(col("Category"))) \
    .withColumn("Genres", trim(col("Genres"))) \
    .withColumn("Reviews", col("Reviews").cast("int")) \
    .withColumn("Rating", col("Rating").cast("float")) \
    .withColumn("Price", regexp_replace("Price", "[$]", "").cast("float")) \
    .withColumn("Installs", regexp_replace("Installs", "[+,]", "").cast("int")) \
    .withColumn("Last Updated", to_date(col("Last Updated"), "MMMM d, yyyy"))


df_cleaned = df_cleaned.dropDuplicates().dropna()
df_cleaned.filter(col("Price") < 0).show()
df_cleaned.filter(col("Reviews") < 0).show()


+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [4]:
# Save cleaned data file for Tableau usage
df_pd = df_cleaned.toPandas()
df_pd.to_csv("cleaned_product_inventory.csv", index=False)

## Data Analysis

### Link to Published Dashboard:
https://public.tableau.com/app/profile/brandon.caudillo/viz/Book1_17469302263580/GooglePlayStoreInsights?publish=yes

## Conclusion

Markdown here

In [6]:
sc.stop()
